# Setting

In [2]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.frequent_patterns import fpgrowth

# 데이터 생성

반찬류와 육류로 임의의 데이터셋 생성  
- 반찬류 : 깻잎, 명이  
- 육류 : 돼지고기, 소고기

In [3]:
dataset = [
	['명이', '소고기'],
	['깻잎', '돼지고기', '소고기'],
	['깻잎', '명이', '돼지고기'],
	['깻잎', '돼지고기'],
	['명이', '돼지고기'],
    ['명이', '소고기', '돼지고기'],
    ['명이', '소고기'],
	['깻잎', '명이', '소고기'],
	['명이', '돼지고기'],
	['명이', '돼지고기', '소고기']
]

In [6]:
# 원-핫 인코딩
te = TransactionEncoder() # 인코더 가져오기
te_array = te.fit(dataset).transform(dataset)
print(te_array)

df = pd.DataFrame(te_array, columns = te.columns_)

[[False False  True  True]
 [ True  True False  True]
 [ True  True  True False]
 [ True  True False False]
 [False  True  True False]
 [False  True  True  True]
 [False False  True  True]
 [ True False  True  True]
 [False  True  True False]
 [False  True  True  True]]


# Apriori

In [7]:
threshold = 0.2

In [8]:
frequent_itemsets = apriori(df, min_support = threshold, use_colnames = True)
print(frequent_itemsets.sort_values('support', ascending = False))

    support         itemsets
2       0.8             (명이)
1       0.7           (돼지고기)
3       0.6            (소고기)
7       0.5       (돼지고기, 명이)
9       0.5        (소고기, 명이)
0       0.4             (깻잎)
4       0.3       (돼지고기, 깻잎)
8       0.3      (돼지고기, 소고기)
5       0.2         (깻잎, 명이)
6       0.2        (소고기, 깻잎)
10      0.2  (돼지고기, 소고기, 명이)


In [9]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)  # 양의 상관관계
rules.sort_values('confidence', ascending = False, inplace = True)  # confidence 순으로 정렬
# print(rules)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
2,(소고기),(명이),0.6,0.8,0.5,0.833333,1.041667,0.02,1.200000,0.100000
1,(깻잎),(돼지고기),0.4,0.7,0.3,0.750000,1.071429,0.02,1.200000,0.111111
3,(명이),(소고기),0.8,0.6,0.5,0.625000,1.041667,0.02,1.066667,0.200000
0,(돼지고기),(깻잎),0.7,0.4,0.3,0.428571,1.071429,0.02,1.050000,0.222222


Lift 값이 1 이상으로 양의 상관관계가 있다고 보이는 조합을 추출  
신뢰도 기준으로 정렬하였을 때 결과는 아래와 같음

1. 소고기 -> 명이
2. 깻잎 -> 돼지고기
3. 명이 -> 소고기
4. 돼지고기 -> 꺳잎

# FP Growth

In [12]:
frequent_itemsets = fpgrowth(df, min_support = threshold, use_colnames=True)
print(frequent_itemsets.sort_values('support', ascending = False))

    support         itemsets
0       0.8             (명이)
2       0.7           (돼지고기)
1       0.6            (소고기)
4       0.5        (소고기, 명이)
7       0.5       (돼지고기, 명이)
3       0.4             (깻잎)
5       0.3      (돼지고기, 소고기)
8       0.3       (돼지고기, 깻잎)
6       0.2  (돼지고기, 소고기, 명이)
9       0.2        (소고기, 깻잎)
10      0.2         (깻잎, 명이)


In [11]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)  # 양의 상관관계
rules.sort_values('confidence', ascending = False, inplace = True)  # confidence 순으로 정렬
# print(rules)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(소고기),(명이),0.6,0.8,0.5,0.833333,1.041667,0.02,1.200000,0.100000
3,(깻잎),(돼지고기),0.4,0.7,0.3,0.750000,1.071429,0.02,1.200000,0.111111
1,(명이),(소고기),0.8,0.6,0.5,0.625000,1.041667,0.02,1.066667,0.200000
2,(돼지고기),(깻잎),0.7,0.4,0.3,0.428571,1.071429,0.02,1.050000,0.222222


Lift 값이 1 이상으로 양의 상관관계가 있다고 보이는 조합을 추출  
신뢰도 기준으로 정렬하였을 때 결과는 아래와 같음

1. 소고기 -> 명이
2. 깻잎 -> 돼지고기
3. 명이 -> 소고기
4. 돼지고기 -> 꺳잎